In [ ]:
import fullcontrol as fc
import numpy as np
import math
from math import tau
from math import pi

import typing as tp

In [ ]:
filename = 'my_design'
# printer options: generic, ultimaker2plus, prusa_i3, ender_3, cr_10, bambulab_x1, toolchanger_T0, toolchanger_T1, toolchanger_T2, toolchanger_T3
print_settings = {
    'extrusion_width': 0.5,
    'extrusion_height': 0.2, 
    'fan_percent': 60,
    'print_speed': 1000,
    'travel_speed': 8000,
    'area_model': 'stadium',
    'e_units': 'mm',
    'dia_feed': 1.75,
    'relative_e': True
    }


gcode_controls = fc.GcodeControls(printer_name='custom', save_as=filename, initialization_data=print_settings, include_date=False)

start_steps = [fc.ManualGcode(text='START_PRINT BED=50 EXTRUDER=200 MESH=last BED_TYPE="Textured PEI Plate"')]

end_steps = [fc.ManualGcode(text='END_PRINT')]


# 'extrusion_width' and 'extrusion_height' are the width and height of the printed line)

# Wavy


In [ ]:
dz = 0.2

ang_res = 256
dtheta = tau/ang_res

height = 20

points = int(height*ang_res/dz)

def radius(z, theta):
    return 0.5*np.sin(3*(z/height)*tau)+5+0.2*np.sin(theta*24)

steps = []
for point in range(points):
    hz = height*point/points
    theta =tau* point/ang_res

    # print(point, hz, theta)

    steps.append(fc.polar_to_point(fc.Point(x=0, y=0, z=hz), radius(hz, theta), theta))

fc.transform(steps, 'plot', fc.PlotControls(color_type='print_sequence'))

# Ribbed

In [ ]:
# TODO: Make a class for objects that modifiers can be applied to

In [ ]:
def smoothed(value, amount, resolution = 2):
    return np.linspace(value-amount, value+amount, resolution)

def polygon(n, z, theta, rad):
    mtheta = theta % (tau / n)
    unitradius = (np.sin(pi * (1/2 - 1/n))) / (np.sin(pi * (1/2 + 1/n) - mtheta))
    return np.mean(rad * unitradius)

def lerp(shape1, shape2, z):
    # z from 0 to 1
    return np.mean(shape1 * (1 - z) + shape2 * z)


In [ ]:
polygon(4, 0, smoothed(0, 0.5), 20)

In [ ]:
dz = 0.2

ribs = 48

ares = ribs*6

dtheta = tau/ares

ddz = dz/ares

height = 50



def innerradius(z, theta):
    return 10

def outerradius(z, theta):
    # return 0.5*np.sin((z/height)*tau)+12#+0.2*np.sin(theta*24)
    return polygon(z, theta + tau * (z /(6* height)), 6, 20)
    # return lerp(polygon(z, theta + tau * (z /(6* height)), 6, 20), polygon(z, theta + tau * (z /(6* height)), 6, 20), z/height)


def wallpoint(radius, theta, hz):
    # Finds point on the outer wall?
    return fc.polar_to_point(fc.Point(x=0, y=0, z=hz), innerradius(hz, theta), theta)
    
steps = [fc.Extruder(on=True)]
hz = 0

for layer in range(int(height/dz)):
    for point in range(ares):

        theta = point * dtheta

        hz = layer * dz + point * ddz

        if point % 6 == 1:
            steps.append(fc.move_polar(steps[-1], centre=steps[-1], radius=outerradius(hz, theta)- innerradius(hz, theta), angle=theta + dtheta/2))
            steps.append(wallpoint(outerradius, theta, hz))
        else:
            steps.append(wallpoint(innerradius, theta, hz))

steps = fc.move(steps, fc.Vector(x=50, y = 50, z=0))

fc.transform(steps, 'plot', fc.PlotControls(color_type='print_sequence'))

# Final framework

In [ ]:
dz = 0.2

ribs = 48

segments = 6

ares = ribs*segments # Segments per rib

dtheta = tau/ares # Angular step

print('minres:', 20*dtheta)

ddz = dz/ares

height = 40

epsilon = 0.05



def innerradius(z, theta):
    return 10

def outerradius(z, theta):
    return polygon(6, z, smoothed(theta + z*tau/(height*6), 0.1), 20)
    # return lerp(polygon(4, z, smoothed(theta, 0.1), 20), polygon(4, z, smoothed(theta + pi/4, 0.1), 20), smoothed(z/height, 1))

def wallpoint(radius, hz, theta, thetarad = None):
    # Finds point on the outer wall?
    if thetarad is None:
        thetarad = theta

    return fc.polar_to_point(fc.Point(x=0, y=0, z=hz), radius(hz, thetarad), theta)
    
steps = [fc.Extruder(on=True)]
hz = 0

total_segments = int(height*ribs*segments/dz)
total_radians = total_segments * dtheta

theta = 0

def segment2z(segment):
    return segment * height / total_segments

def theta_to_z(theta):
    return theta * height / total_radians

def ribTheta(radius, epsilon):
    return epsilon / (2 * radius)

for segment in range(total_segments):
    theta = segment * dtheta
    z = segment2z(segment)
    steps.append(wallpoint(innerradius, z, theta))

    if segment % segments == 0:

        midtheta = theta + ribTheta(innerradius(z, theta), epsilon)
        endtheta = theta + 2 * ribTheta(innerradius(z, theta), epsilon)
        deltheta =  ribTheta(outerradius(z, theta), epsilon)

        steps.append(wallpoint(outerradius, theta_to_z(midtheta - deltheta), midtheta - deltheta, midtheta))
        steps.append(wallpoint(outerradius, theta_to_z(midtheta + deltheta), midtheta + deltheta, midtheta))
        steps.append(wallpoint(innerradius, theta_to_z(endtheta), endtheta))





steps = fc.move(steps, fc.Vector(x=50, y = 50, z=0))

fc.transform(steps, 'plot', fc.PlotControls(color_type='print_sequence', style='line'))

In [ ]:


total = start_steps+steps+end_steps
fc.transform(total, 'gcode', gcode_controls)
